In [13]:
import tensorflow as tf
import tensorflow_text as tf_text

In [2]:
!pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [22]:
class TextPreprocessor(tf.keras.layers.Layer):
    def __init__(self, max_tokens=10000, **kwargs):
        super().__init__(**kwargs)
        self.tokenizer = tf.keras.layers.TextVectorization(max_tokens=max_tokens)

    def adapt(self, texts):
        self.tokenizer.adapt(texts)

    def preprocess(self, text):
        return self.tokenizer(text)

    def call(self, text):
        return self.preprocess(text)

In [23]:
class CosineSimilarityLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        tfidf_manual, tfidf_keyword = inputs
        similarity = tf.keras.losses.cosine_similarity(tfidf_manual, tfidf_keyword)
        return (1 + similarity) * 50  # Convert to percentage

In [24]:
def create_similarity_model():
    manual_abstract_input = tf.keras.Input(shape=(), dtype=tf.string, name='manual_abstract')
    keyword_query_input = tf.keras.Input(shape=(), dtype=tf.string, name='keyword_query')

    text_preprocessor = TextPreprocessor()
    tfidf_manual = text_preprocessor(manual_abstract_input)
    tfidf_keyword = text_preprocessor(keyword_query_input)

    similarity = CosineSimilarityLayer()([tfidf_manual, tfidf_keyword])

    model = tf.keras.Model(inputs=[manual_abstract_input, keyword_query_input], outputs=similarity)
    return model

In [25]:
# Create the similarity model
model = create_similarity_model()

In [36]:
# Example usage
manual_abstract = ["In the modern technology industry, the utilization of machine learning and natural language processing is increasingly dominant, with applications like facial recognition leveraging machine learning algorithms to understand and interpret users' natural language."]
keyword_queries = ["machine learning OR natural language processing"]  # Ensure it's a list of lists

In [37]:
# Adapt the vectorizer on the manual abstract
preprocessor_layer = model.get_layer(index=2)
preprocessor_layer.adapt([manual_abstract])

In [40]:
# Example usage
manual_abstract = ["In the modern technology industry, the utilization of machine learning and natural language processing is increasingly dominant, with applications like facial recognition leveraging machine learning algorithms to understand and interpret users' natural language."]
keyword_queries = ["machine learning OR natural language processing"]

# Calculate similarity
similarities = model.predict([manual_abstract[0], keyword_queries[0]])  # Ambil elemen pertama dari masing-masing daftar
print("Similarities:", similarities)

ValueError: Unrecognized data type: x=["In the modern technology industry, the utilization of machine learning and natural language processing is increasingly dominant, with applications like facial recognition leveraging machine learning algorithms to understand and interpret users' natural language.", 'machine learning OR natural language processing'] (of type <class 'list'>)

In [ ]:
# Export the model
model_save_path = 'similarity_model'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
# Loading the model for later use
loaded_model = tf.keras.models.load_model(model_save_path, custom_objects={'TextPreprocessor': TextPreprocessor, 'CosineSimilarityLayer': CosineSimilarityLayer})

In [ ]:
# Calculate similarity with loaded model
similarities_loaded = loaded_model.predict([manual_abstract, keyword_queries])
print("Similarities with loaded model:", similarities_loaded)